Import library

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter

In [2]:
k_mer = 6

fullDataSet = pd.read_csv('AllerginDataFinal.csv')
dataFrame = pd.DataFrame(fullDataSet)

totalData = len(dataFrame)
# for i in range(totalData):
#     if dataFrame.loc[i,'isAllergen'] == -1:
#         dataFrame.at[i,'isAllergen'] = 0

print(dataFrame.head())
print(dataFrame.tail())
print(totalData)

   Unnamed: 0                                           Sequence  isAllergen
0           0  AVTPMSADQLALFKSSWNTVKHNEVDILYAVFKANPDIQAKFPQFA...           1
1           1  MSWQAYVDDHLMCDVGDGNTPASAAIIGHDGSVWAQSANFPQLKPE...           1
2           2  ITCGQVSSNLAPCINYVKGGGAVPPACCNGIRNVNNLARTTADRRA...           1
3           3  MAISTAMKLSLVALLCIVVALPIAQAITCGQVASSISPCVNYVKSG...           1
4           4  MKLCILAVAVFVVAVSAQGPPPLPPFVANAPPAVQAEFRQLANGAP...           1
      Unnamed: 0                                           Sequence  \
4424        2422  AVDLYTQAITMSPKNPELFADRAQANIKLNYFTEAVVDANKAIELD...   
4425        2423  MALSSTSTTNSLLPNRSLVQNQPLLPSPLKNAFFSNNSTKTVRFVQ...   
4426        2424                                        VLLPEYGGTQV   
4427        2425  QQIPRDVQQCCNQLEQIQDPQCRCEGLMKVVQQEEQTGKVQGRQRQ...   
4428        2426  MADQLTEDQISEFKEAFSLFDKDGDGCITTKELGTVMRSLGQNPTE...   

      isAllergen  
4424          -1  
4425          -1  
4426          -1  
4427          -1  
4428          -1

Method fod creating kmers


In [3]:
def getKMers(sequence, ksize):
    kmers = []
    n_kmers = len(sequence) - ksize + 1

    for i in range(n_kmers):
        kmer = sequence[i:i + ksize]
        kmers.append(kmer)
    return kmers

generate kmers for allergen

In [4]:
len_seq = []
for row in range(totalData) :
  sequence = dataFrame.loc[row, "Sequence"]
  kMers = getKMers(sequence, k_mer)
  dataFrame.loc[row, 'kmers'] = ' '.join(kMers)
  len_seq.append(len(sequence))

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 200000, dtype=np.float64) #max_features = 10000
kmersCorpus = dataFrame['kmers']
X = vectorizer.fit_transform(kmersCorpus).toarray()
y = dataFrame['isAllergen']

In [6]:
for i in range(X.shape[0]):
    X[i] /= (len_seq[i] - k_mer + 1)
X[np.isnan(X)] = 0
print(np.any(np.isnan(X)))
print(set(X[0]))

/var/folders/6_/xv_wk6qn5k9g5yztczrn0zlh0000gn/T/ipykernel_1032/1196936956.py:2: RuntimeWarning: invalid value encountered in true_divide
  X[i] /= (len_seq[i] - k_mer + 1)


False
{0.0, 0.00684931506849315}


Splitting the dataset into the Training set and Test set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
print(set(X_train[10]))

{0.0, 0.006896551724137931}


SVM

In [8]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC()

In [9]:
from sklearn.metrics import confusion_matrix, accuracy_score
svm_y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, svm_y_pred)
print(cm)
accuracy_score(y_test, svm_y_pred)

[[199  12]
 [ 44 188]]


0.873589164785553

In [10]:
svmScore = clf.decision_function(X)
dataFrame['svmScore'] = svmScore
print(dataFrame.head())

   Unnamed: 0                                           Sequence  isAllergen  \
0           0  AVTPMSADQLALFKSSWNTVKHNEVDILYAVFKANPDIQAKFPQFA...           1   
1           1  MSWQAYVDDHLMCDVGDGNTPASAAIIGHDGSVWAQSANFPQLKPE...           1   
2           2  ITCGQVSSNLAPCINYVKGGGAVPPACCNGIRNVNNLARTTADRRA...           1   
3           3  MAISTAMKLSLVALLCIVVALPIAQAITCGQVASSISPCVNYVKSG...           1   
4           4  MKLCILAVAVFVVAVSAQGPPPLPPFVANAPPAVQAEFRQLANGAP...           1   

                                               kmers  svmScore  
0  AVTPMS VTPMSA TPMSAD PMSADQ MSADQL SADQLA ADQL...  0.942371  
1  MSWQAY SWQAYV WQAYVD QAYVDD AYVDDH YVDDHL VDDH...  1.032991  
2  ITCGQV TCGQVS CGQVSS GQVSSN QVSSNL VSSNLA SSNL...  0.999735  
3  MAISTA AISTAM ISTAMK STAMKL TAMKLS AMKLSL MKLS...  1.000249  
4  MKLCIL KLCILA LCILAV CILAVA ILAVAV LAVAVF AVAV... -0.266419  


In [11]:
# from sklearn.naive_bayes import GaussianNB
# clfNb = GaussianNB()
# clfNb.fit(X_train, y_train)
# y_predNb = clfNb.predict(X)
# dataFrame['nbScore'] = y_predNb

In [12]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.linear_model import LinearRegression
# clfNB = LinearRegression()
# clfNB.fit(X_train, y_train)

# y_pred = clfNB.predict(X_test)
# # cm = confusion_matrix(y_test, y_pred)
# # print(cm)
# # accuracy_score(y_test, y_pred)
# print(y_pred)

In [13]:
# regressionScore = clfNB.score(X_test, y_test)
# # dataFrame['regScore'] = regressionScore
# # print(dataFrame.head())
# # print(dataFrame.tail())
# print(regressionScore)

In [14]:
print(dataFrame.head(n=2))
for i in range(5):
    dataFrame['E' + str(i+1)] = 0.0
print(dataFrame.head(n=2))

   Unnamed: 0                                           Sequence  isAllergen  \
0           0  AVTPMSADQLALFKSSWNTVKHNEVDILYAVFKANPDIQAKFPQFA...           1   
1           1  MSWQAYVDDHLMCDVGDGNTPASAAIIGHDGSVWAQSANFPQLKPE...           1   

                                               kmers  svmScore  
0  AVTPMS VTPMSA TPMSAD PMSADQ MSADQL SADQLA ADQL...  0.942371  
1  MSWQAY SWQAYV WQAYVD QAYVDD AYVDDH YVDDHL VDDH...  1.032991  
   Unnamed: 0                                           Sequence  isAllergen  \
0           0  AVTPMSADQLALFKSSWNTVKHNEVDILYAVFKANPDIQAKFPQFA...           1   
1           1  MSWQAYVDDHLMCDVGDGNTPASAAIIGHDGSVWAQSANFPQLKPE...           1   

                                               kmers  svmScore   E1   E2   E3  \
0  AVTPMS VTPMSA TPMSAD PMSADQ MSADQL SADQLA ADQL...  0.942371  0.0  0.0  0.0   
1  MSWQAY SWQAYV WQAYVD QAYVDD AYVDDH YVDDHL VDDH...  1.032991  0.0  0.0  0.0   

    E4   E5  
0  0.0  0.0  
1  0.0  0.0  


In [15]:
# Calculate E1~5 values from sequences
acids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
Es= [
    [0.008, 0.171, 0.255, 0.303, -0.132, 0.149, 0.221, 0.218, 0.023, -0.353, -.267, .243, -0.239, -0.329, .173, .199, .068, -0.296, -0.141, -0.274],
    [0.134, -0.361, 0.038, -0.057, 0.174, -0.184, -0.28, 0.562, -0.177, 0.071, 0.018, -0.339, -0.141, -0.023, 0.286, 0.238, 0.147, -0.186, -0.057, 0.136],
    [-0.475, 0.107, 0.117, -0.014, 0.07, 0.03, -0.315, -0.024, 0.041, -0.088, -0.265, -0.044, -0.155, 0.072, 0.407, -0.015, -0.015, 0.389, 0.425, -0.187],
    [-0.039, -0.258, 0.118, 0.225, 0.565, 0.035, 0.157, 0.018, 0.28, -0.195, -0.274, -0.325, 0.321, -0.002, -0.215, -0.068, -0.132, 0.083, -0.096, -0.196],
    [0.181, -0.364, -0.055, 0.156, -0.374, -0.112, 0.303, 0.106, -0.021, -0.107, 0.206, -0.027, 0.077, 0.208, 0.384, -0.196, -0.274, 0.297, -0.091, -0.299]
]

def calc(seq, sequenceIndex):
    for e in range(0, 5):
        total = 0.0
        for acidIdx in range(len(acids)):
            c = acids[acidIdx]
            cnt = seq.count(c)
            total += cnt * Es[e][acidIdx]
        dataFrame.at[sequenceIndex,'E'+str(e+1)] = total

for sequenceIndex in range(len(dataFrame)):
    calc(dataFrame.iloc[sequenceIndex]["Sequence"], sequenceIndex)

print(dataFrame.head(n = 2))
print(dataFrame.tail(n = 2))

   Unnamed: 0                                           Sequence  isAllergen  \
0           0  AVTPMSADQLALFKSSWNTVKHNEVDILYAVFKANPDIQAKFPQFA...           1   
1           1  MSWQAYVDDHLMCDVGDGNTPASAAIIGHDGSVWAQSANFPQLKPE...           1   

                                               kmers  svmScore     E1      E2  \
0  AVTPMS VTPMSA TPMSAD PMSADQ MSADQL SADQLA ADQL...  0.942371  1.180   2.805   
1  MSWQAY SWQAYV WQAYVD QAYVDD AYVDDH YVDDHL VDDH...  1.032991  3.133  11.204   

       E3     E4     E5  
0 -11.197 -7.499  5.731  
1  -8.119 -4.692  4.117  
      Unnamed: 0                                           Sequence  \
4427        2425  QQIPRDVQQCCNQLEQIQDPQCRCEGLMKVVQQEEQTGKVQGRQRQ...   
4428        2426  MADQLTEDQISEFKEAFSLFDKDGDGCITTKELGTVMRSLGQNPTE...   

      isAllergen                                              kmers  svmScore  \
4427          -1  QQIPRD QIPRDV IPRDVQ PRDVQQ RDVQQC DVQQCC VQQC... -0.709649   
4428          -1  MADQLT ADQLTE DQLTED QLTEDQ LTEDQI TEDQIS E

In [16]:
for i in range(8):
    dataFrame['D' + str(i+1)] = 0.0
print(dataFrame.head(n=2))

   Unnamed: 0                                           Sequence  isAllergen  \
0           0  AVTPMSADQLALFKSSWNTVKHNEVDILYAVFKANPDIQAKFPQFA...           1   
1           1  MSWQAYVDDHLMCDVGDGNTPASAAIIGHDGSVWAQSANFPQLKPE...           1   

                                               kmers  svmScore     E1      E2  \
0  AVTPMS VTPMSA TPMSAD PMSADQ MSADQL SADQLA ADQL...  0.942371  1.180   2.805   
1  MSWQAY SWQAYV WQAYVD QAYVDD AYVDDH YVDDHL VDDH...  1.032991  3.133  11.204   

       E3     E4     E5   D1   D2   D3   D4   D5   D6   D7   D8  
0 -11.197 -7.499  5.731  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1  -8.119 -4.692  4.117  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  


In [17]:
# Calculate D1~8 values from sequences
Ds= [
[0.651, 1.272, 0.965, 0.972, 0.885, 1.068, 1.075, 0.548, 1.133, 0.958, 0.958, 1.068, 1.09, 1.207, 0.841, 0.768, 0.87, 1.492, 1.324, 0.856],
[0.973, 1.261, 1.393, 1.562, 0.661, 1.261, 1.477, 1.081, 1.249, 0.625, 0.589, 1.357, 0.685, 0.625, 0.961, 1.105, 1.033, 0.649, 0.745, 0.709],
[0.614, 0.6, 0.063, 0.466, 1.072, 0, 0.473, 0.071, 0.614, 2.222, 1.531, 1.157, 1.178, 2.025, 1.954, 0.049, 0.049, 2.66, 1.884, 1.319],
[0.57, 2.052, 1.437, 1.14, 0.433, 1.619, 1.117, 0.525, 0.981, 0.41, 0.525, 2.212, 0.707, 0.547, 1.14, 1.003, 1.072, 0.73, 1.368, 0.41],
[0.78, 1.58, 1.2, 1.35, 0.55, 1.19, 1.45, 0.68, 0.99, 0.47, 0.56, 1.1, 0.66, 0.47, 0.69, 1, 1.05, 0.7, 1, 0.51],
[0.3634, 1.9266, 0.9274, 0.8575, 0.6683, 0.997, 1.1128, 0.2343, 1.0631, 0.9845, 1.1486, 1.5369, 1.0385, 1.3731, 0.5536, 0.4656, 0.6918, 2.3415, 1.6385, 0.7066],
[0.443, 0.798, 0.497, 0.429, 0.499, 0.584, 0.404, 0.542, 0.759, 0.582, 0.521, 0.812, 0.461, 0.579, 0.487, 0.391, 0.485, 0.641, 0.662, 0.51],
[0.233, 0.313, 0.296, 0.37, 0.253, 0.381, 0.326, 0.322, 0.274, 0.266, 0.337, 0.334, 0.294, 0.271, 0.291, 0.281, 0.307, 0.297, 0.284, 0.298]
]

def calc(seq, sequenceIndex):
    for e in range(0, 8):
        total = 0.0
        for acidIdx in range(len(acids)):
            c = acids[acidIdx]
            cnt = seq.count(c)
            total += cnt * Ds[e][acidIdx]
        dataFrame.at[sequenceIndex,'D'+str(e+1)] = total

for sequenceIndex in range(len(dataFrame)):
    calc(dataFrame.iloc[sequenceIndex]["Sequence"], sequenceIndex)

print(dataFrame.head(n = 2))
print(dataFrame.tail(n = 2))

   Unnamed: 0                                           Sequence  isAllergen  \
0           0  AVTPMSADQLALFKSSWNTVKHNEVDILYAVFKANPDIQAKFPQFA...           1   
1           1  MSWQAYVDDHLMCDVGDGNTPASAAIIGHDGSVWAQSANFPQLKPE...           1   

                                               kmers  svmScore     E1      E2  \
0  AVTPMS VTPMSA TPMSAD PMSADQ MSADQL SADQLA ADQL...  0.942371  1.180   2.805   
1  MSWQAY SWQAYV WQAYVD QAYVDD AYVDDH YVDDHL VDDH...  1.032991  3.133  11.204   

       E3     E4     E5       D1       D2       D3       D4      D5        D6  \
0 -11.197 -7.499  5.731  143.180  153.931  142.874  143.452  133.35  142.1784   
1  -8.119 -4.692  4.117  118.919  133.124  112.737  118.193  112.88  111.2361   

       D7      D8  
0  80.919  45.545  
1  69.721  40.458  
      Unnamed: 0                                           Sequence  \
4427        2425  QQIPRDVQQCCNQLEQIQDPQCRCEGLMKVVQQEEQTGKVQGRQRQ...   
4428        2426  MADQLTEDQISEFKEAFSLFDKDGDGCITTKELGTVMRSLGQNPTE...  

In [18]:
X_ann = dataFrame[['E1', 'E2', 'E3', 'E4', 'E5','svmScore']]
y_ann = y
X_train_ann, X_test_ann, y_train_ann, y_test_ann = train_test_split(X_ann, y_ann, test_size=0.33, random_state=42)

In [19]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(4,2,5), activation='relu', solver='adam', max_iter=1000)
mlp.fit(X_train_ann, y_train_ann)

MLPClassifier(hidden_layer_sizes=(4, 2, 5), max_iter=1000)

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
predict_ann__ = mlp.predict(X_test_ann)

cm = confusion_matrix(y_test_ann, predict_ann__)
print(cm)
accuracy_score(y_test_ann, predict_ann__)

[[728  41]
 [ 94 599]]


0.9076607387140903

In [42]:
best_acc = 0
best_spec = (0, 0, 0)
bestMlp = MLPClassifier()

for i in range(1, 15):
    for j in range(1, 15):
        for k in range(1, 15):
            mlp = MLPClassifier(hidden_layer_sizes=(i,j,k), activation='relu', solver='adam', max_iter=1000)
            mlp.fit(X_train_ann, y_train_ann)
            predict_ann__ = mlp.predict(X_test_ann)
            
            acc = accuracy_score(y_test_ann, predict_ann__)
            if acc > best_acc:
                best_acc = acc
                best_spec = (i, j, k)
                cm = confusion_matrix(y_test_ann, predict_ann__)
                bestMlp = mlp
print(cm)
print(best_spec, best_acc)

[[738  31]
 [ 43 650]]
(4, 7, 2) 0.9493844049247606


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# for n in range(5):
strat_k_fold = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)
ac = cross_val_score(bestMlp, X = X, y = y, cv = strat_k_fold)
print(ac.mean())